In [2]:
import os, re, csv
import pandas as pd
from eunjeon import Mecab
from tqdm import tqdm
import gensim
#import soynlp
mecab = Mecab()

eta_df = pd.read_csv("질문.csv", encoding='utf8')

In [3]:
from sqlalchemy import create_engine
engine = create_engine('sqlite://', echo=False)

In [4]:
eta_df.to_sql('eta_assis', con=engine)

In [5]:
query = """
select 내용, id
from eta_assis
where 내용 LIKE '%이중%' or 내용 LIKE '%부전%' 
   or 내용 LIKE '%전공심화%' or 내용 LIKE '%전심%' 
   or 내용 LIKE '%융합%' or 내용 LIKE '%융합전공%'
"""


In [6]:
dm_eta_df = pd.read_sql_query(query, con=engine)

In [7]:
dm_eta_df.head(5)

,내용,id
0,안녕하세요 조교님! 혹시 부전공 신청은 막학기에 불가능한가요??ㅜ이번학기가 막학기고...,112883775
1,안녕하세요 제가 지금 부전으로 경영이 되어 있는데이번 5월 변경신청기간에 이중으로 ...,112519890
2,안녕하세요 조교님이중 변경을 5월 달에 하면 2학기부터 변경된 이중으로 강의를 듣는...,112461913
3,안녕하세요 조교님!제가 맨 처음에 이중전공을 gbt로 신청했다가이중전공 경영으로 변...,112132727
4,이중전공을 바꾸고싶은데 혹시 바꾸게 된다면 그전에 받은 학점은 이어가는 건가요? 아...,112064305


In [8]:
content = dm_eta_df['내용'].tolist()
ids = dm_eta_df['id'].tolist()

In [9]:
DM = [0]*len(ids) #double major
SM = [0]*len(ids) #sub major
CM = [0]*len(ids) #convergence major
IC = [0]*len(ids) #intensified course

In [10]:
for i, doc in enumerate(content):
    if '이중' in doc:
        DM[i] = 1
    if '부전' in doc:
        SM[i] = 1
    if '전공심화' in doc or '전심' in doc:
        CM[i] = 1
    if '융합' in doc or '융합전공' in doc:
        IC[i] = 1

In [11]:
raw_data = {'이중전공': DM,
            '부전공': SM,
            '전공심화': CM,
           '융합전공': IC,
           'id': ids}

data = pd.DataFrame(raw_data)

In [12]:
data.head(10)

,이중전공,부전공,전공심화,융합전공,id
0,1,1,0,0,112883775
1,1,1,0,0,112519890
2,1,1,0,0,112461913
3,1,1,0,0,112132727
4,1,0,0,0,112064305
5,1,0,0,0,111800950
6,1,1,0,0,111649639
7,0,1,0,0,111097777
8,1,0,0,0,111086527
9,0,1,0,0,110891116


In [13]:
data.to_sql('eta_mjr', con=engine)

In [14]:
query = """
select a.내용, m.이중전공, m.부전공, m.전공심화, m.융합전공, m.id
from eta_assis a, eta_mjr m
where a.id = m.id
"""

In [15]:
res = pd.read_sql_query(query, con=engine)

In [16]:
res.head(5)

,내용,이중전공,부전공,전공심화,융합전공,id
0,안녕하세요 조교님! 혹시 부전공 신청은 막학기에 불가능한가요??ㅜ이번학기가 막학기고...,1,1,0,0,112883775
1,안녕하세요 제가 지금 부전으로 경영이 되어 있는데이번 5월 변경신청기간에 이중으로 ...,1,1,0,0,112519890
2,안녕하세요 조교님이중 변경을 5월 달에 하면 2학기부터 변경된 이중으로 강의를 듣는...,1,1,0,0,112461913
3,안녕하세요 조교님!제가 맨 처음에 이중전공을 gbt로 신청했다가이중전공 경영으로 변...,1,1,0,0,112132727
4,이중전공을 바꾸고싶은데 혹시 바꾸게 된다면 그전에 받은 학점은 이어가는 건가요? 아...,1,0,0,0,112064305


In [17]:
res.to_csv('에타_전공분류.csv', encoding = 'utf8')